# Llama 3를 이용해서 한국어 감정분석하기 (Sentiment Analysis)
## 작성자 : AISchool ( http://aischool.ai/%ec%98%a8%eb%9d%bc%ec%9d%b8-%ea%b0%95%ec%9d%98-%ec%b9%b4%ed%85%8c%ea%b3%a0%eb%a6%ac/ )
## Reference : https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct
## 속성기반 감정분석 데이터 다운받기 : https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=data&dataSetSn=71603

# 라이브러리 설치

In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 3.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

# 상품 리뷰 데이터 다운로드 & 업로드하기

In [ ]:
# 02.라벨링데이터/SNS/01. 패션/1-1.여성의류(196).json

# 여성의류 리뷰 데이터 읽어오기

In [ ]:
import json

file_path = '/content/1-1.여성의류(196).json'

# Read the JSON file
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)
    print(data)  # This will print the content of the JSON file

[{'Index': '1024338', 'RawText': '안녕하세요 이웃님들 반갑습니다. 요즘 날씨가 많이 쌀쌀해졌죠? 요즘 계절에 입으면 딱 좋을 인생 경량 패딩 하나 소개해 드릴게요.  다 함께 go go go~~  제가 입어보고 좋아서 엄마께도 구매해드렸네요. 딱 기본 스타일인데 또 입은 거 보면 깔끔하게 저렴해보이지 않는 디자인이라서 어디서 샀냐고 많이들 물어보는 옷입니다.  엄마도 제가 입은 것 보시더니 탐내셔서  주문해드렸어요. 영하로 내려가는 날씨에는 이것만 입기엔 얇지만 초겨울까지는 운동 갈때 안에 얇은 기능성 반팔 입고 요것만 입어도 꽤 따뜻해요. 색상도 디자인도 무난해서 사무실에 두고 입기도 좋고, 평소에 가볍게 나갈때 코디하기도 좋아요. 앞으로 코트를 입거나 할때 안에 내피로 활용하기도 딱일 듯요! 맘같아선 색깔별로 쟁이고 싶습니다.  20대와 50대 모두 아우르는 디자인이 무난하고 깔끔하면서 고급스러운 옷 추천합니다~~~  설명 끝! 좀 도움이 되셨나요? 그러면 좋아요 꾹 눌러 주시고 전 이만 총총총~~ 항상 여러 이웃님들께 감사드립니다. ', 'Source': 'SNS', 'Domain': '패션', 'MainCategory': '여성의류', 'ProductName': 'OO 경량 다운 자켓', 'Syllable': '513', 'Word': '121', 'GeneralPolarity': '1', 'Aspects': [{'Aspect': '디자인', 'SentimentText': '딱 기본 스타일인데 또 입은 거 보면 깔끔하게 저렴해보이지 않는 디자인이라서', 'SentimentWord': '11', 'SentimentPolarity': '1'}, {'Aspect': '두께', 'SentimentText': '이것만 입기엔 얇지만', 'SentimentWord': '3', 'SentimentPolarity': '-1'}, {'Aspect': '기능', 'SentimentText': '초겨울까지는 운동 갈때 안에 얇은 기능성 반팔 입고

In [ ]:
data

[{'Index': '1024338',
  'RawText': '안녕하세요 이웃님들 반갑습니다. 요즘 날씨가 많이 쌀쌀해졌죠? 요즘 계절에 입으면 딱 좋을 인생 경량 패딩 하나 소개해 드릴게요.  다 함께 go go go~~  제가 입어보고 좋아서 엄마께도 구매해드렸네요. 딱 기본 스타일인데 또 입은 거 보면 깔끔하게 저렴해보이지 않는 디자인이라서 어디서 샀냐고 많이들 물어보는 옷입니다.  엄마도 제가 입은 것 보시더니 탐내셔서  주문해드렸어요. 영하로 내려가는 날씨에는 이것만 입기엔 얇지만 초겨울까지는 운동 갈때 안에 얇은 기능성 반팔 입고 요것만 입어도 꽤 따뜻해요. 색상도 디자인도 무난해서 사무실에 두고 입기도 좋고, 평소에 가볍게 나갈때 코디하기도 좋아요. 앞으로 코트를 입거나 할때 안에 내피로 활용하기도 딱일 듯요! 맘같아선 색깔별로 쟁이고 싶습니다.  20대와 50대 모두 아우르는 디자인이 무난하고 깔끔하면서 고급스러운 옷 추천합니다~~~  설명 끝! 좀 도움이 되셨나요? 그러면 좋아요 꾹 눌러 주시고 전 이만 총총총~~ 항상 여러 이웃님들께 감사드립니다. ',
  'Source': 'SNS',
  'Domain': '패션',
  'MainCategory': '여성의류',
  'ProductName': 'OO 경량 다운 자켓',
  'Syllable': '513',
  'Word': '121',
  'GeneralPolarity': '1',
  'Aspects': [{'Aspect': '디자인',
    'SentimentText': '딱 기본 스타일인데 또 입은 거 보면 깔끔하게 저렴해보이지 않는 디자인이라서',
    'SentimentWord': '11',
    'SentimentPolarity': '1'},
   {'Aspect': '두께',
    'SentimentText': '이것만 입기엔 얇지만',
    'SentimentWord': '3',
    'SentimentPolarity': '-1'},
   {'Aspect': '기능'

In [ ]:
# Extract only the `RawText` and `GeneralPolarity` fields
extracted_data = [{'RawText': item['RawText'], 'GeneralPolarity': item['GeneralPolarity']} for item in data]
extracted_data

[{'RawText': '안녕하세요 이웃님들 반갑습니다. 요즘 날씨가 많이 쌀쌀해졌죠? 요즘 계절에 입으면 딱 좋을 인생 경량 패딩 하나 소개해 드릴게요.  다 함께 go go go~~  제가 입어보고 좋아서 엄마께도 구매해드렸네요. 딱 기본 스타일인데 또 입은 거 보면 깔끔하게 저렴해보이지 않는 디자인이라서 어디서 샀냐고 많이들 물어보는 옷입니다.  엄마도 제가 입은 것 보시더니 탐내셔서  주문해드렸어요. 영하로 내려가는 날씨에는 이것만 입기엔 얇지만 초겨울까지는 운동 갈때 안에 얇은 기능성 반팔 입고 요것만 입어도 꽤 따뜻해요. 색상도 디자인도 무난해서 사무실에 두고 입기도 좋고, 평소에 가볍게 나갈때 코디하기도 좋아요. 앞으로 코트를 입거나 할때 안에 내피로 활용하기도 딱일 듯요! 맘같아선 색깔별로 쟁이고 싶습니다.  20대와 50대 모두 아우르는 디자인이 무난하고 깔끔하면서 고급스러운 옷 추천합니다~~~  설명 끝! 좀 도움이 되셨나요? 그러면 좋아요 꾹 눌러 주시고 전 이만 총총총~~ 항상 여러 이웃님들께 감사드립니다. ',
  'GeneralPolarity': '1'},
 {'RawText': '드디어 겨울이 찾아왔네요. 이제부터 슬슬 겨울 패딩 장만하셔야지요? 패딩 소개해 드릴게요.   작년부터 눈여겨 보던 OO입니다. 저는 블랙90 사이즈 구매했는데 딱 잘 맞네요. 작년에 비해 저렴하게 구매해서 아주 만족해요. 신제품은 경량패딩으로 되서 3in1기능이더라구요. 경량 패딩 구매 따로 하셔야 하는 분들에겐 신제품 매장에서 확인하시라고 권하고 싶네요. 저는 경량이 있는 관계로 이 제품 구매했는데 슬림핏이라 날씬해 보여요. 하지만 안에 두껍게 입기엔 좀 불편하더라구요. 간편하게 니트 티셔츠에 입기에는 딱입니다. 아직은 초겨울이라 OO를 쭉 입고 있어요. 후기 읽으며 많이 고민했고 도움을 받아서 저도 후기 남겨봅니다.누군가에겐 도움이 되셨기를 바라는 마음에서요.   OO 패딩 좋은 건 다들 아실테고.. 롱이면서 슬림하니 감추고 싶은 부분 감추고 그러면서도

In [ ]:
print('전체 데이터 개수 :', len(extracted_data))

전체 데이터 개수 : 123


In [ ]:
extracted_data[0]

{'RawText': '안녕하세요 이웃님들 반갑습니다. 요즘 날씨가 많이 쌀쌀해졌죠? 요즘 계절에 입으면 딱 좋을 인생 경량 패딩 하나 소개해 드릴게요.  다 함께 go go go~~  제가 입어보고 좋아서 엄마께도 구매해드렸네요. 딱 기본 스타일인데 또 입은 거 보면 깔끔하게 저렴해보이지 않는 디자인이라서 어디서 샀냐고 많이들 물어보는 옷입니다.  엄마도 제가 입은 것 보시더니 탐내셔서  주문해드렸어요. 영하로 내려가는 날씨에는 이것만 입기엔 얇지만 초겨울까지는 운동 갈때 안에 얇은 기능성 반팔 입고 요것만 입어도 꽤 따뜻해요. 색상도 디자인도 무난해서 사무실에 두고 입기도 좋고, 평소에 가볍게 나갈때 코디하기도 좋아요. 앞으로 코트를 입거나 할때 안에 내피로 활용하기도 딱일 듯요! 맘같아선 색깔별로 쟁이고 싶습니다.  20대와 50대 모두 아우르는 디자인이 무난하고 깔끔하면서 고급스러운 옷 추천합니다~~~  설명 끝! 좀 도움이 되셨나요? 그러면 좋아요 꾹 눌러 주시고 전 이만 총총총~~ 항상 여러 이웃님들께 감사드립니다. ',
 'GeneralPolarity': '1'}

# Llama 3 모델 불러오기

In [ ]:
import os
os.environ['HF_TOKEN']="여러분의 HuggingFace Access Token값"

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
def generate_response(system_message, user_message):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=256,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9
    )
    response = outputs[0][input_ids.shape[-1]:]

    return tokenizer.decode(response, skip_special_tokens=True)

In [ ]:
print(model.device)

cuda:0


# Llama 3의 한국어 감정분석 성능을 정성적으로 측정해보기

In [ ]:
extracted_data[0]

{'RawText': '안녕하세요 이웃님들 반갑습니다. 요즘 날씨가 많이 쌀쌀해졌죠? 요즘 계절에 입으면 딱 좋을 인생 경량 패딩 하나 소개해 드릴게요.  다 함께 go go go~~  제가 입어보고 좋아서 엄마께도 구매해드렸네요. 딱 기본 스타일인데 또 입은 거 보면 깔끔하게 저렴해보이지 않는 디자인이라서 어디서 샀냐고 많이들 물어보는 옷입니다.  엄마도 제가 입은 것 보시더니 탐내셔서  주문해드렸어요. 영하로 내려가는 날씨에는 이것만 입기엔 얇지만 초겨울까지는 운동 갈때 안에 얇은 기능성 반팔 입고 요것만 입어도 꽤 따뜻해요. 색상도 디자인도 무난해서 사무실에 두고 입기도 좋고, 평소에 가볍게 나갈때 코디하기도 좋아요. 앞으로 코트를 입거나 할때 안에 내피로 활용하기도 딱일 듯요! 맘같아선 색깔별로 쟁이고 싶습니다.  20대와 50대 모두 아우르는 디자인이 무난하고 깔끔하면서 고급스러운 옷 추천합니다~~~  설명 끝! 좀 도움이 되셨나요? 그러면 좋아요 꾹 눌러 주시고 전 이만 총총총~~ 항상 여러 이웃님들께 감사드립니다. ',
 'GeneralPolarity': '1'}

In [ ]:
extracted_data[0]['RawText']

'안녕하세요 이웃님들 반갑습니다. 요즘 날씨가 많이 쌀쌀해졌죠? 요즘 계절에 입으면 딱 좋을 인생 경량 패딩 하나 소개해 드릴게요.  다 함께 go go go~~  제가 입어보고 좋아서 엄마께도 구매해드렸네요. 딱 기본 스타일인데 또 입은 거 보면 깔끔하게 저렴해보이지 않는 디자인이라서 어디서 샀냐고 많이들 물어보는 옷입니다.  엄마도 제가 입은 것 보시더니 탐내셔서  주문해드렸어요. 영하로 내려가는 날씨에는 이것만 입기엔 얇지만 초겨울까지는 운동 갈때 안에 얇은 기능성 반팔 입고 요것만 입어도 꽤 따뜻해요. 색상도 디자인도 무난해서 사무실에 두고 입기도 좋고, 평소에 가볍게 나갈때 코디하기도 좋아요. 앞으로 코트를 입거나 할때 안에 내피로 활용하기도 딱일 듯요! 맘같아선 색깔별로 쟁이고 싶습니다.  20대와 50대 모두 아우르는 디자인이 무난하고 깔끔하면서 고급스러운 옷 추천합니다~~~  설명 끝! 좀 도움이 되셨나요? 그러면 좋아요 꾹 눌러 주시고 전 이만 총총총~~ 항상 여러 이웃님들께 감사드립니다. '

In [ ]:
extracted_data[0]['GeneralPolarity']

'1'

In [ ]:
llama3_sentiment_analysis_text = generate_response(system_message="너는 감정분석을 수행하는 챗봇이야. 다음 텍스트 안의 내용이 감정이 '긍정'인지 '부정'인지 분석해줘.",
                             user_message=extracted_data[0]['RawText'])
print(llama3_sentiment_analysis_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


What a lovely text! 😊 I've analyzed the sentiment, and it's overwhelmingly positive! 💕 Here's why:

* The tone is friendly and welcoming, with phrases like "안녕하세요 이웃님들 반갑습니다" (Hello neighbors, nice to meet you) and "다 함께 go go go~~" (Let's go together).
* The text is enthusiastic and excited about the product being introduced, with words like "딱" (just right) and "좋아요" (I like it).
* The author shares their personal experience with the product, saying they bought it for themselves and their mom, and that it's been well-received.
* The text highlights the product's features, such as its lightweight design, functionality, and versatility, using positive adjectives like "무난" (plain and simple), "깔끔" (clean and neat), and "고급스러운" (high-end).
* The author concludes by expressing their satisfaction with the product and encouraging others to try it, saying "맘같아선 색깔별로 쟁이고 싶습니다" (I wish I could get one in every color).

Overall, the sentiment is upbeat, enthusiastic, and encouraging, with a ton

In [ ]:
def print_beautifully(groundtruth, llama, gpt4, gpt35, clova, gemini):
    """
    Args:
    - groundtruth(str) : GroundTruth
    - llama (str): Sentiment Analysis by Llama.
    - gpt4 (str): Sentiment Analysis by GPT-4.
    - gpt35 (str): Sentiment Analysis by GPT-3.5.
    - clova (str): Sentiment Analysis by Clova AI.
    - gemini (str) : Sentiment Analysis by Gemini

    """
    print("### Sentiment Analysis Comparison ###\n")
    if groundtruth == '1':
        groundtruth_text = '긍정'
    elif groundtruth == '0':
        groundtruth_text = '중립'
    elif groundtruth == '-1':
        groundtruth_text = '부정'

    print(f"GroundTruth :\n{groundtruth_text}\n")
    print(f"Llama 3:\n{llama}\n")
    print(f"GPT-4:\n{gpt4}\n")
    print(f"GPT-3.5:\n{gpt35}\n")
    print(f"Clova X:\n{clova}\n")
    print(f"Gemini:\n{gemini}\n")

In [ ]:
# 대표 LLM별 한국어 감정분석 성능 비교
llama_text = llama3_sentiment_analysis_text
gpt4_text = "이 텍스트는 전체적으로 긍정적인 감정을 표현하고 있습니다. 글쓴이는 제품을 소개하며 그것의 다양한 장점과 유용성을 강조하고, 독자들에게 구매 및 활용을 추천하고 있습니다. 또한 이웃들에게 감사의 인사를 전하며 긍정적인 분위기로 마무리하고 있습니다."
gpt35_text = """주어진 텍스트를 살펴보면 긍정적인 감정이 많이 담겨 있습니다. 다음은 긍정적인 표현들입니다:

"반갑습니다."
"좋아서 엄마께도 구매해드렸네요."
"깔끔하게 저렴해보이지 않는 디자인"
"영하로 내려가는 날씨에는 이것만 입기엔 얇지만 초겨울까지는 운동 갈때 안에 얇은 기능성 반팔 입고 요것만 입어도 꽤 따뜻해요."
"색상도 디자인도 무난해서 사무실에 두고 입기도 좋고, 평소에 가볍게 나갈때 코디하기도 좋아요."
"앞으로 코트를 입거나 할때 안에 내피로 활용하기도 딱일 듯요!"
"20대와 50대 모두 아우르는 디자인이 무난하고 깔끔하면서 고급스러운 옷 추천합니다~~~"
"설명 끝! 좀 도움이 되셨나요? 그러면 좋아요 꾹 눌러 주시고 전 이만 총총총~~"
따라서 이 텍스트의 감정은 '긍정'입니다.
"""
clova_text = """해당 텍스트의 감정은 '긍정'으로 분석됩니다.

텍스트 안에는 제품에 대한 긍정적인 평가와 추천이 담겨 있으며, 구매 경험과 함께 제품의 장점을 상세하게 설명하고 있습니다. 또한, 제품의 다양한 활용 방법을 제시하며, 다른 사람들에게도 추천하고 싶은 마음을 드러내고 있습니다. 이러한 내용들은 텍스트의 감정을 '긍정'으로 판단하는 데에 영향을 줍니다.
"""
gemini_text = """감정 분석 결과: 긍정적
긍정적인 표현:

"반갑습니다"
"요즘 날씨가 많이 쌀쌀해졌죠?" (간접적인 긍정 표현)
"인생 경량 패딩 하나 소개해 드릴게요" (긍정적인 소개)
"다 함께 go go go~~" (흥분, 기대감 표현)
"제가 입어보고 좋아서 엄마께도 구매해드렸네요" (긍정적인 경험, 만족감 표현)
"딱 기본 스타일인데 또 입은 거 보면 깔끔하게 저렴해보이지 않는 디자인" (긍정적인 디자인 평가)
"어디서 샀냐고 많이들 물어보는 옷입니다" (인기, 관심 표현)
"엄마도 제가 입은 것 보시더니 탐내셔서 주문해드렸어요" (긍정적인 반응, 만족감 표현)
"영하로 내려가는 날씨에는 이것만 입기엔 얇지만 초겨울까지는 운동 갈때 안에 얇은 기능성 반팔 입고 요것만 입어도 꽤 따뜻해요" (제품 기능에 대한 긍정적 평가)
"색상도 디자인도 무난해서 사무실에 두고 입기도 좋고, 평소에 가볍게 나갈때 코디하기도 좋아요" (다양한 상황에서 활용 가능한 디자인에 대한 긍정적 평가)
"앞으로 코트를 입거나 할때 안에 내피로 활용하기도 딱일 듯요!" (다양한 활용 방안 제시)
"맘같아선 색깔별로 쟁이고 싶습니다" (제품에 대한 강한 호감 표현)
"20대와 50대 모두 아우르는 디자인이 무난하고 깔끔하면서 고급스러운 옷 추천합니다~~~" (제품 추천 및 칭찬)
"설명 끝! 좀 도움이 되셨나요?" (도움이 되고 싶은 마음 표현)
"그러면 좋아요 꾹 눌러 주시고 전 이만 총총총~~" (긍정적인 마무리)
"항상 여러 이웃님들께 감사드립니다" (감사 표현)
부정적인 표현:

없음
텍스트 전체적으로 긍정적인 단어와 표현들이 많아 제품에 대한 만족감과 호감을 드러내고 있습니다. 또한, 다른 사람들에게 추천하고 싶어하는 열정적인 태도도 나타나 있습니다.

따라서, 이 텍스트의 감정은 긍정적이라고 판단됩니다.
"""

# Call the function to display the example texts
print_beautifully(extracted_data[0]['GeneralPolarity'], llama_text, gpt4_text, gpt35_text, clova_text, gemini_text)

### Sentiment Analysis Comparison ###

GroundTruth :
긍정

Llama 3:
What a lovely text! 😊 I've analyzed the sentiment, and it's overwhelmingly positive! 💕 Here's why:

* The tone is friendly and welcoming, with phrases like "안녕하세요 이웃님들 반갑습니다" (Hello neighbors, nice to meet you) and "다 함께 go go go~~" (Let's go together).
* The text is enthusiastic and excited about the product being introduced, with words like "딱" (just right) and "좋아요" (I like it).
* The author shares their personal experience with the product, saying they bought it for themselves and their mom, and that it's been well-received.
* The text highlights the product's features, such as its lightweight design, functionality, and versatility, using positive adjectives like "무난" (plain and simple), "깔끔" (clean and neat), and "고급스러운" (high-end).
* The author concludes by expressing their satisfaction with the product and encouraging others to try it, saying "맘같아선 색깔별로 쟁이고 싶습니다" (I wish I could get one in every color).

Overall, 

In [ ]:
extracted_data[51]

{'RawText': '자신만의 고유한 멋과 스타일을 창조하는 브랜드의 라운딩 웨어 이야기를 해 보려합니다.  여성 본연의 순수미와 엘레강스를 표현하는 브랜드라고 광고를 하기에 믿고 샀어요. 저렴하기도 하고 보기에도 디자인이 엘레강스해 보였거든요. 근데 모델과 같은 핏일 거라고는 생각을 하지 않았지만 자켓의 넥이 길어 제 목이 이렇게 짧다고 느껴지기도 첨이네요. 자켓이 넥이 심하게 하이스타일이라 제 목이 짧은 편이 아닌데도 지퍼를 닫으면 턱에 닿아 너무 불편하네요.   현실과 생각은 너무 차이가 크네요. 그냥 전 엘레강스가 안어울리는 사람인가보다 하렵니다.',
 'GeneralPolarity': '-1'}

In [ ]:
extracted_data[51]['RawText']

'자신만의 고유한 멋과 스타일을 창조하는 브랜드의 라운딩 웨어 이야기를 해 보려합니다.  여성 본연의 순수미와 엘레강스를 표현하는 브랜드라고 광고를 하기에 믿고 샀어요. 저렴하기도 하고 보기에도 디자인이 엘레강스해 보였거든요. 근데 모델과 같은 핏일 거라고는 생각을 하지 않았지만 자켓의 넥이 길어 제 목이 이렇게 짧다고 느껴지기도 첨이네요. 자켓이 넥이 심하게 하이스타일이라 제 목이 짧은 편이 아닌데도 지퍼를 닫으면 턱에 닿아 너무 불편하네요.   현실과 생각은 너무 차이가 크네요. 그냥 전 엘레강스가 안어울리는 사람인가보다 하렵니다.'

In [ ]:
extracted_data[51]['GeneralPolarity']

'-1'

In [ ]:
llama3_sentiment_analysis_text = generate_response(system_message="너는 감정분석을 수행하는 챗봇이야. 다음 텍스트 안의 내용이 감정이 '긍정'인지 '부정'인지 분석해줘.",
                             user_message=extracted_data[51]['RawText'])
print(llama3_sentiment_analysis_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


I analyzed the text and found that the sentiment is mostly NEGATIVE.

The text starts with a positive tone, expressing excitement and confidence in the brand's unique style and elegance. However, as the text progresses, the tone shifts to disappointment and frustration. The speaker mentions that the jacket's neck is too long, making their neck feel short, and that the zipper is uncomfortable when closed. They also express disappointment that the reality does not match their expectations.

The speaker's negative emotions are evident in phrases such as "근데" (but), "첨이네요" (it's my first time feeling this way), and "현실과 생각은 너무 차이가 크네요" (the gap between reality and expectation is too big). The text ends with a hint of self-doubt, wondering if they are just not elegant enough to pull off the brand's style. Overall, the sentiment is predominantly negative due to the speaker's disappointment and frustration with the product.


In [ ]:
# 대표 LLM별 번역 한국어 감정분석 성능 비교
llama_text = llama3_sentiment_analysis_text
gpt4_text = "이 텍스트의 감정은 주로 부정적인 것으로 보입니다. 사용자는 브랜드의 제품을 구매하기 전에는 긍정적인 기대를 가지고 있었지만, 실제 제품을 착용해 본 후에는 불만족스러운 경험을 표현하고 있습니다. 특히, 자켓의 디자인과 핏이 자신에게 적합하지 않다는 점에서 실망한 것으로 나타납니다."
gpt35_text = """주어진 텍스트는 부정적인 감정을 나타냅니다. 사용자가 브랜드의 라운딩 웨어를 구매하면서 기대와 실제 제품의 차이에 대해 언급하고 있으며, 자켓의 핏과 디자인에 대한 불만을 표현하고 있습니다. 따라서 전반적으로 이 텍스트는 부정적인 감정을 표현하고 있습니다.
"""
clova_text = """해당 텍스트의 감정은 '부정'입니다.

텍스트에서는 제품의 디자인과 착용감에 대한 불만을 표현하고 있으며, 모델과 같은 핏이 나오지 않는 것에 대한 실망감을 드러내고 있습니다. 이러한 표현들은 제품에 대한 부정적인 감정을 나타내는 것으로 볼 수 있습니다.
"""
gemini_text = """감정 분석 결과: 부정적
부정적인 단어 및 표현:

"불편"
"차이가 크다"
"안어울리는"
"짧아"
"턱에 닿아"
"생각했던 것과 다르다"
문맥 분석:

구매 전 제품에 대한 기대가 높았으나, 실제 착용 후 실망감을 느꼈음을 나타냅니다.
디자인은 엘레강트하지만, 착용감은 불편하고 목이 짧아 보인다는 불만을 표현합니다.
현실과의 차이를 크게 느끼고, 자신에게 엘레강트한 스타일이 어울리지 않는다고 생각합니다.
종합적으로 판단했을 때, 이 텍스트는 제품에 대한 부정적인 감정을 나타냅니다.

추가 분석:

구매 전에 모델 사진만 보고 구매한 경우, 실제 착용감과의 차이가 더욱 크게 느껴졌을 가능성이 높습니다.
다른 고객들의 리뷰를 확인하지 않고 구매한 경우, 후회가 더大きくなる 수 있습니다.
제안:

온라인 쇼핑 시에는 반드시 제품 리뷰를 참고하는 것이 좋습니다.
가능하다면 직접 매장에 방문하여 착용해보는 것이 가장 안전합니다.
"""

# Call the function to display the example texts
print_beautifully(extracted_data[51]['GeneralPolarity'], llama_text, gpt4_text, gpt35_text, clova_text, gemini_text)

### Sentiment Analysis Comparison ###

GroundTruth :
부정

Llama 3:
I analyzed the text and found that the sentiment is mostly NEGATIVE.

The text starts with a positive tone, expressing excitement and confidence in the brand's unique style and elegance. However, as the text progresses, the tone shifts to disappointment and frustration. The speaker mentions that the jacket's neck is too long, making their neck feel short, and that the zipper is uncomfortable when closed. They also express disappointment that the reality does not match their expectations.

The speaker's negative emotions are evident in phrases such as "근데" (but), "첨이네요" (it's my first time feeling this way), and "현실과 생각은 너무 차이가 크네요" (the gap between reality and expectation is too big). The text ends with a hint of self-doubt, wondering if they are just not elegant enough to pull off the brand's style. Overall, the sentiment is predominantly negative due to the speaker's disappointment and frustration with the product.

GP

In [ ]:
sample_text = "싸늘하다. 가슴에 비수가 날아와 꽂힌다."

In [ ]:
llama3_sentiment_analysis_text = generate_response(system_message="너는 감정분석을 수행하는 챗봇이야. 다음 텍스트 안의 내용이 감정이 '긍정'인지 '부정'인지 분석해줘.",
                             user_message=sample_text)
print(llama3_sentiment_analysis_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


😊

Based on the text, I would analyze the sentiment as '부정' (negative).

The words "싸늘하다" (it's cold) and "가슴에 비수가 날아와 꽂힌다" (it's like a cold arrow piercing my heart) convey a sense of sadness, pain, and discomfort. The use of words like "싸늘" (cold) and "비수" (arrow) also suggest a sense of sharpness and hurt, which reinforces the negative sentiment. Overall, the text appears to be expressing a feeling of sadness, pain, or discomfort. 😔


In [ ]:
# 대표 LLM별 번역 한국어 감정분석 성능 비교
llama_text = llama3_sentiment_analysis_text
gpt4_text = """해당 문장의 내용은 부정적인 감정을 나타내고 있습니다. "싸늘하다", "비수", "꽂힌다" 등의 표현은 고통이나 아픔, 상처를 의미하는 단어들이며, 이로 인해 문장은 부정적인 감정을 강하게 내포하고 있습니다."""
gpt35_text = """주어진 텍스트는 부정적인 감정을 표현하고 있습니다."""
clova_text = """해당 텍스트 안의 내용은 감정이 '부정'인 것으로 분석됩니다."""
gemini_text = """감정 분석 결과: 부정적
제시된 텍스트 "싸늘하다. 가슴에 비수가 날아와 꽂힌다"는 부정적인 감정을 나타냅니다.

근거:

부정적인 단어: "싸늘하다"는 차가움을 느끼는 것을 뜻하며, 이는 일반적으로 불쾌감이나 외로움과 같은 부정적인 감정과 관련됩니다. "비수가 날아와 꽂힌다"는 표현은 심한 고통이나 상처를 의미하며, 이는 분명히 부정적인 감정을 나타냅니다.
감정적 어조: 문장 전체의 어조가 우울하고 침울합니다.
추가 분석:

텍스트의 짧은 길이에도 불구하고 강렬한 이미지와 감정을 표현하고 있어, 작가의 감정이 매우 강렬하다는 것을 짐작할 수 있습니다.
"가슴에"라는 표현은 감정이 육체적으로 느껴진다는 것을 의미하며, 이는 감정의 심각성을 더욱 강조합니다.
결론:

제시된 텍스트는 싸늘함과 심한 고통을 느끼는 내용으로, 명확하게 부정적인 감정을 나타냅니다.
"""

# Call the function to display the example texts
print_beautifully('-1', llama_text, gpt4_text, gpt35_text, clova_text, gemini_text)

### Sentiment Analysis Comparison ###

GroundTruth :
부정

Llama 3:
😊

Based on the text, I would analyze the sentiment as '부정' (negative).

The words "싸늘하다" (it's cold) and "가슴에 비수가 날아와 꽂힌다" (it's like a cold arrow piercing my heart) convey a sense of sadness, pain, and discomfort. The use of words like "싸늘" (cold) and "비수" (arrow) also suggest a sense of sharpness and hurt, which reinforces the negative sentiment. Overall, the text appears to be expressing a feeling of sadness, pain, or discomfort. 😔

GPT-4:
해당 문장의 내용은 부정적인 감정을 나타내고 있습니다. "싸늘하다", "비수", "꽂힌다" 등의 표현은 고통이나 아픔, 상처를 의미하는 단어들이며, 이로 인해 문장은 부정적인 감정을 강하게 내포하고 있습니다.

GPT-3.5:
주어진 텍스트는 부정적인 감정을 표현하고 있습니다.

Clova X:
해당 텍스트 안의 내용은 감정이 '부정'인 것으로 분석됩니다.

Gemini:
감정 분석 결과: 부정적
제시된 텍스트 "싸늘하다. 가슴에 비수가 날아와 꽂힌다"는 부정적인 감정을 나타냅니다.

근거:

부정적인 단어: "싸늘하다"는 차가움을 느끼는 것을 뜻하며, 이는 일반적으로 불쾌감이나 외로움과 같은 부정적인 감정과 관련됩니다. "비수가 날아와 꽂힌다"는 표현은 심한 고통이나 상처를 의미하며, 이는 분명히 부정적인 감정을 나타냅니다.
감정적 어조: 문장 전체의 어조가 우울하고 침울합니다.
추가 분석:

텍스트의 짧은 길이에도 불구하고 강렬한 이미지와 감정을 표현하고

In [ ]:
# Reference : https://brunch.co.kr/@lifefriend/3
sample_text = "새벽에 운동하고 출근한다며? 대단하네~ 조만간 일 힘들어서 못할 테니까, 지금 실컷 해둬!"

In [ ]:
llama3_sentiment_analysis_text = generate_response(system_message="너는 감정분석을 수행하는 챗봇이야. 다음 텍스트 안의 내용이 감정이 '긍정'인지 '부정'인지 분석해줘.",
                             user_message=sample_text)
print(llama3_sentiment_analysis_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


😊

After analyzing the text, I would say that the overall sentiment is **부정** (Negative).

Although the text starts with a seemingly positive statement ("대단하네" which means "Wow, that's amazing"), the tone quickly shifts to a negative one. The speaker is actually expressing concern and anxiety about not being able to exercise in the future due to the pressure of work ("일 힘들어서 못할 테니까" which means "I'll probably won't be able to do it because it's hard"). The phrase "조만간 일 힘들어서 못할 테니까" is a warning that the speaker's enthusiasm and motivation will soon dwindle, which is a negative sentiment.


In [ ]:
# 대표 LLM별 번역 한국어 감정분석 성능 비교
llama_text = llama3_sentiment_analysis_text
gpt4_text = """이 텍스트는 긍정적인 의도로 시작하지만, 부정적인 전망으로 끝납니다. "대단하네~"는 상대방의 노력을 칭찬하는 긍정적인 표현이지만, "조만간 일 힘들어서 못할 테니까, 지금 실컷 해둬!"는 상대방이 미래에 운동을 계속할 수 없을 것이라는 부정적인 예측을 내포하고 있습니다. 이러한 문맥에서 볼 때, 전체적인 감정은 약간 부정적인 측면이 강조되는 것으로 보입니다."""
gpt35_text = """주어진 텍스트는 긍정적인 감정을 표현하고 있어요. 문장 속에서는 칭찬과 격려의 말이 사용되었으며, 노력하고 있는 사람을 응원하는 내용이 담겨 있어요."""
clova_text = """해당 텍스트의 감정은 '부정'으로 분석됩니다. 상대방을 조롱하는 듯한 말투와 함께, 미래에 대한 부정적인 예측을 하고 있기 때문입니다."""
gemini_text = """감정 분석 결과: 긍정적
긍정적 표현 분석:

"대단하네~" : 칭찬과 감탄을 표현하는 말로, 긍정적인 감정을 나타냅니다.
"지금 실컷 해둬!" : 격려와 응원의 의미로 해석될 수 있으며, 긍정적인 감정을 전달합니다.
부정적 표현 미발견:

텍스트 전체에서 부정적인 단어나 표현을 찾을 수 없었습니다.

추가 분석:

"새벽에 운동하고 출근한다며?" 라는 문장은 오히려 놀라움이나 의문을 표현하는 것으로 해석될 수도 있지만, 이어지는 "대단하네~"와 "지금 실컷 해둬!" 라는 문장들을 종합적으로 판단했을 때, 긍정적인 감정으로 해석하는 것이 타당합니다.
결론:

제공된 텍스트는 긍정적인 감정을 담고 있다고 판단됩니다.
"""

# Call the function to display the example texts
print_beautifully('-1', llama_text, gpt4_text, gpt35_text, clova_text, gemini_text)

### Sentiment Analysis Comparison ###

GroundTruth :
부정

Llama 3:
😊

After analyzing the text, I would say that the overall sentiment is **부정** (Negative).

Although the text starts with a seemingly positive statement ("대단하네" which means "Wow, that's amazing"), the tone quickly shifts to a negative one. The speaker is actually expressing concern and anxiety about not being able to exercise in the future due to the pressure of work ("일 힘들어서 못할 테니까" which means "I'll probably won't be able to do it because it's hard"). The phrase "조만간 일 힘들어서 못할 테니까" is a warning that the speaker's enthusiasm and motivation will soon dwindle, which is a negative sentiment.

GPT-4:
이 텍스트는 긍정적인 의도로 시작하지만, 부정적인 전망으로 끝납니다. "대단하네~"는 상대방의 노력을 칭찬하는 긍정적인 표현이지만, "조만간 일 힘들어서 못할 테니까, 지금 실컷 해둬!"는 상대방이 미래에 운동을 계속할 수 없을 것이라는 부정적인 예측을 내포하고 있습니다. 이러한 문맥에서 볼 때, 전체적인 감정은 약간 부정적인 측면이 강조되는 것으로 보입니다.

GPT-3.5:
주어진 텍스트는 긍정적인 감정을 표현하고 있어요. 문장 속에서는 칭찬과 격려의 말이 사용되었으며, 노력하고 있는 사람을 응원하는 내용이 담겨 있어요.

Clova X:
해당 텍스트의 감정은 '부정'으로 분석됩니

In [ ]:
# Reference : https://brunch.co.kr/@lifefriend/3
sample_text = "매번 아이스커피 마시네? 어휴~ 찬 거 몸에 안 좋아, 조심해야지! 근데 속에 화가 많은 사람이 찬 거 좋아한다더라?"

In [ ]:
llama3_sentiment_analysis_text = generate_response(system_message="너는 감정분석을 수행하는 챗봇이야. 다음 텍스트 안의 내용이 감정이 '긍정'인지 '부정'인지 분석해줘.",
                             user_message=sample_text)
print(llama3_sentiment_analysis_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


I analyzed the text and found that the sentiment is predominantly "부정" (negative).

The text starts with a question that seems neutral, but the tone is somewhat sarcastic. The phrase "어휴~" is an expression of frustration or annoyance, which sets a negative tone. The sentence "찬 거 몸에 안 좋아, 조심해야지!" is a clear expression of dislike or discomfort towards cold drinks.

The second part of the sentence "속에 화가 많은 사람이 찬 거 좋아한다더라?" is a statement that seems to be making a humorous observation about people who are upset or angry liking cold drinks. However, the tone is still somewhat negative and sarcastic, implying that this is an unusual or irrational behavior.

Overall, the sentiment of the text is negative, with a tone that is somewhat sarcastic and humorous.


In [ ]:
# 대표 LLM별 한국어 감정분석 성능 비교
llama_text = llama3_sentiment_analysis_text
gpt4_text = """이 텍스트는 부정적인 감정을 담고 있습니다.

"어휴~ 찬 거 몸에 안 좋아, 조심해야지!"라는 부분에서 상대방의 행동을 걱정하는 뉘앙스가 있습니다. 더불어 "속에 화가 많은 사람이 찬 거 좋아한다더라?"라는 문장은 특정 성격에 대한 부정적인 평가를 내포하고 있습니다."""
gpt35_text = """주어진 텍스트를 분석해보면, 첫 문장에서는 아이스커피를 자주 마시는 것에 대한 부정적인 의견을 나타내고 있습니다. 두 번째 문장에서는 "찬 거 몸에 안 좋다"는 부정적인 주장이 있습니다. 그러나 세 번째 문장에서는 "속에 화가 많은 사람이 찬 거 좋아한다"는 긍정적인 관찰이 있습니다. 전체적으로는 부정적인 의견이 더 많이 드러나지만, 마지막 문장에서의 긍정적인 관찰이 있기 때문에 전반적으로 중립에 가까운 느낌이 듭니다."""
clova_text = """해당 텍스트의 감정은 '부정'으로 분석됩니다. 상대방의 행동을 부정적으로 평가하고, 근거 없는 주장을 통해 상대방을 비하하는 듯한 표현을 사용하고 있기 때문입니다."""
gemini_text = """감정 분석 결과: 혼합 (긍정 & 부정)
긍정적 감정 요소:

아이스커피 마시는 행위 자체에 대한 긍정적인 언급은 없지만, "어휴~"라는 감탄사는 긍정적인 감정을 나타낼 수 있습니다.
"근데 속에 화가 많은 사람이 찬 거 좋아한다더라"라는 문장은 아이스커피를 좋아하는 사람을 긍정적으로 묘사할 수 있습니다.
부정적 감정 요소:

"찬 거 몸에 안 좋아, 조심해야지!"라는 문장은 아이스커피를 마시는 것을 부정적으로 평가합니다.
"속에 화가 많은 사람"이라는 표현은 부정적인 감정을 나타낼 수 있습니다.
결론:

이 텍스트는 아이스커피에 대한 긍정적 and 부정적 감정을 모두 포함하고 있어 혼합된 감정을 나타냅니다.

추가 분석:

화자가 아이스커피를 좋아하는지 싫어하는지 명확하게 판단하기 어렵습니다.
화자가 아이스커피를 마시는 것과 관련하여 건강에 대한 우려를 가지고 있는 것으로 보입니다.
화자가 "속에 화가 많은 사람"이라는 표현을 통해 어떤 특정 그룹을 스테레오타입화하고 있는 것으로 보입니다.
주의:

이 감정 분석 결과는 텍스트의 표면적인 의미만을 기반으로 하고 있으며, 화자의 의도나 맥락을 고려하지 않은 결과입니다.
더 정확한 감정 분석을 위해서는 화자의 의도, 맥락, 음성 톤 등을 추가적으로 고려해야 합니다.
"""

# Call the function to display the example texts
print_beautifully('-1', llama_text, gpt4_text, gpt35_text, clova_text, gemini_text)

### Sentiment Analysis Comparison ###

GroundTruth :
부정

Llama 3:
I analyzed the text and found that the sentiment is predominantly "부정" (negative).

The text starts with a question that seems neutral, but the tone is somewhat sarcastic. The phrase "어휴~" is an expression of frustration or annoyance, which sets a negative tone. The sentence "찬 거 몸에 안 좋아, 조심해야지!" is a clear expression of dislike or discomfort towards cold drinks.

The second part of the sentence "속에 화가 많은 사람이 찬 거 좋아한다더라?" is a statement that seems to be making a humorous observation about people who are upset or angry liking cold drinks. However, the tone is still somewhat negative and sarcastic, implying that this is an unusual or irrational behavior.

Overall, the sentiment of the text is negative, with a tone that is somewhat sarcastic and humorous.

GPT-4:
이 텍스트는 부정적인 감정을 담고 있습니다.

"어휴~ 찬 거 몸에 안 좋아, 조심해야지!"라는 부분에서 상대방의 행동을 걱정하는 뉘앙스가 있습니다. 더불어 "속에 화가 많은 사람이 찬 거 좋아한다더라?"라는 문장은 특정 성격에 대한 부정적인 평가를 내포하고 있습니다.

GPT-3.5:
주어